In [ ]:
import pandas as pd
from maquette import *

In [ ]:
data = Dataset("fraud-analysis").get()

In [ ]:
data.head(5)

In [ ]:
# Create two dataframes with fraud and non-fraud data 
df_fraud = data.loc[data.fraud == 1] 
df_non_fraud = data.loc[data.fraud == 0]

In [ ]:
# Count the fraudulent records
df_fraud.fraud.count()

In [ ]:
# Find in which activities fraud occurs the most
data.groupby('category')['amount','fraud'].mean().sort_values(by=['amount'],ascending=False)

In [ ]:
# Find in which ages, fraud occurs the most, mostly in younger ages <18
data.groupby('age')['amount','fraud'].mean().sort_values(by=['amount'],ascending=False)

In [ ]:
#There is only one unique zipCode values so we drop them
data_reduced = data.drop(['zipcodeOri','zipMerchant'],axis=1)

In [ ]:
data_reduced

In [ ]:
data_reduced.columns

In [ ]:
col_categorical = data_reduced.select_dtypes(include= ['object']).columns

In [ ]:
col_categorical

In [ ]:
# turning object columns type to categorical for easing the transformation process
for col in col_categorical:
    data_reduced[col] = data_reduced[col].astype('category')

In [ ]:
#.cat.codes assigns numbers to categories
data_reduced[col_categorical] = data_reduced[col_categorical].apply(lambda x: x.cat.codes)

In [ ]:
data_reduced

In [ ]:
#Define our independent variable (X) and dependant/target variable y
X = data_reduced.drop(['fraud'],axis=1)
y = data['fraud']

In [ ]:
X

In [ ]:
y

In [ ]:
#if a fixed value is assigned like random_state = 42 then no matter how many times you execute your code the result would be the same i.e, same values in train and test datasets.
import imblearn
#sm = imblearn.over_sampling.SMOTE(random_state=42)

In [ ]:
sm

In [ ]:
X_res, y_res = sm.fit_resample(X, y)
y_res = pd.DataFrame(y_res)
#print(y_res[0].value_counts())

In [ ]:
y_res.count()

In [ ]:
y_res.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.3,random_state=42,shuffle=True,stratify=y_res)

In [ ]:
# %% K-Neighbours Classifier
import sklearn.neighbors
#from sklearn.model_selection import train_test_split
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5,p=1)

knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)


print("Classification Report for K-Nearest Neighbours: \n", 
sklearn.metrics.classification_report(y_test, y_pred))
print("Confusion Matrix of K-Nearest Neigbours: \n", 
sklearn.metrics.confusion_matrix(y_test,y_pred))
#plot_roc_auc(y_test, knn.predict_proba(X_test)[:,1])

In [ ]:
# %% Random Forest Classifier
import sklearn.ensemble

rf_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=100,max_depth=8,random_state=42,
                                verbose=1,class_weight="balanced")

rf_clf.fit(X_train,y_train)
y_pred = rf_clf.predict(X_test)

print("Classification Report for Random Forest Classifier: \n", sklearn.metrics.classification_report(y_test, y_pred))
print("Confusion Matrix of Random Forest Classifier: \n", sklearn.metrics.confusion_matrix(y_test,y_pred))
